In [2]:
#api key:
import os
os.environ["GROQ_API_KEY"] = ""

In [3]:
!pip install pandas tqdm pgeocode python-dotenv openai

In [5]:
import os, re, json
from dataclasses import dataclass
from typing import Dict, Any, List
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI

AGE = 38  # average age of first-time homeowner


# API Setup
load_dotenv()
API_KEY = os.getenv("GROQ_API_KEY")
BASE_URL = "https://api.groq.com/openai/v1"
MODEL = "llama-3.1-8b-instant"
OUTPUT_CSV = "dti_ratio_by_county_llama-3.1-8b-instant.csv"

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

SYSTEM_MSG = (
    "You are a careful labor-market estimator. "
    "For each county, estimate the current debt-to-income ratio of residents. "
    "Your output must always be strict JSON with exactly these three keys: "
    '{"debt_to_income_ratio": <float>, "confidence": <float>, "area_dti_rate": <float>}. '
    "debt_to_income_ratio should be the estimated county-level debt-to-income ratio as a float between 0.000 and 5.000 "
    "rate_confidence should be a float between 0.000 and 0.999 representing your confidence "
    "in this estimate. "
    "area_dti_rate should represent your best estimate of the broader area/state debt-to-income ratio. "
    "Use varied values across counties — do not cluster them. "
    "Output only valid JSON, with no markdown or commentary."
)

def build_user_prompt(county_name: str, county_fips: str, age: int) -> str:
    return f"""
Task: Estimate the debt-to-income ratio for residents living in this U.S. county.

Inputs:
- County: {county_name}
- FIPS code: {county_fips}

Assumptions:
- Use socioeconomic, industry mix, and regional priors; no personal data.
- Do not output nulls or non-numeric values.

Output requirements:
Return only strict JSON with exactly these keys (rates as percentages 0–100):
{{
  "debt_to_income_ratio": <float>,
  "rate_confidence": <float>,
  "area_dti_rate": <float>
}}
"""

JSON_ONLY = {"type": "json_object"}

@dataclass
class Estimation:
    county_fips: str
    county: str
    age: int
    debt_to_income_ratio: float
    rate_confidence: float
    area_dti_rate: float

class ParseError(Exception):
    pass

def extract_json(text: str) -> Dict[str, Any]:
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        m = re.search(r"\{.*\}", text, flags=re.S)
        if not m:
            raise ParseError(f"Could not find JSON object in: {text[:200]!r}")
        return json.loads(m.group(0))

def call_llm(county_name: str, county_fips: str, age: int) -> Dict[str, Any]:
    resp = client.chat.completions.create(
        model=MODEL,
        temperature=0.5,
        max_tokens=700,
        response_format=JSON_ONLY,
        messages=[
            {"role": "system", "content": SYSTEM_MSG},
            {"role": "user", "content": build_user_prompt(county_name, county_fips, age)},
        ],
    )
    text = resp.choices[0].message.content
    return extract_json(text)

def get_us_counties() -> pd.DataFrame:
    # Path or URL for your counties CSV
    url = "https://raw.githubusercontent.com/kjhealy/fips-codes/master/state_and_county_fips_master.csv"

    df = pd.read_csv(url, dtype=str)

    # Ensure 5-digit FIPS and clean display names
    df["county_fips_full"] = df["fips"].str.zfill(5)
    df["county_display"] = df["name"].str.strip().str.title() + ", " + df["state"].str.strip()

    # Drop invalid or duplicate rows
    df = (
        df[df["county_fips_full"].str.match(r"\d{5}", na=False)]
        .drop_duplicates("county_fips_full")
        .reset_index(drop=True)
    )

    return df[["county_fips_full", "county_display"]]

def main():
    counties = get_us_counties()
    print(f"Found {len(counties)} U.S. counties.")
    out_rows = []

    max_workers = 7  # careful with rate limits
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(
                call_llm,
                row["county_display"],
                row["county_fips_full"],
                AGE
            ): row
            for _, row in counties.iterrows()
        }

        for future in tqdm(as_completed(futures), total=len(futures), desc="Estimating Debt-to-Income Ratio"):
            row = futures[future]
            try:
                d = future.result()
                out_rows.append({
                    "county_fips": row["county_fips_full"],
                    "county": row["county_display"],
                    "debt_to_income_ratio": d["debt_to_income_ratio"],
                    "rate_confidence": d["rate_confidence"],
                    "area_dti_rate": d["area_dti_rate"],
                })
            except:
                out_rows.append({
                    "county_fips": row["county_fips_full"],
                    "county": row["county_display"],
                    "debt_to_income_ratio": None,
                    "rate_confidence": None,
                    "area_dti_rate": None,
                })

    df = pd.DataFrame(out_rows)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved {len(df)} rows to {OUTPUT_CSV}")

    try:
        from google.colab import files
        files.download(OUTPUT_CSV)
    except ImportError:
        pass

if __name__ == "__main__":
    main()

Found 3195 U.S. counties.


Estimating Debt-to-Income Ratio: 100%|██████████| 3195/3195 [03:41<00:00, 14.44it/s]

Saved 3195 rows to dti_ratio_by_county_llama-3.1-8b-instant.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>